In [1]:
!cd

D:\Users\Ray\Desktop\FakeNews\notebook


In [5]:
import pickle
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import os
import io
import numpy as np
import pandas as pd
import subprocess
import torch
import random
import shutil
from torchvision import transforms
from torchvision import datasets
from tqdm import tqdm

In [6]:
img_path = r'..\weibo\images'
img_list = os.listdir(img_path)
output_dir = r'D:\Users\Ray\Desktop\FakeNews\img'
img_list

['00081a88gw1e0k0fzv6nkj.jpg',
 '00081a88jw1e3eymq2471j.jpg',
 '00081a88jw1e5bxst6kncj20gs0llgok.jpg',
 '00081a88jw1e8aiei4m2ej20cg0fsgnk.jpg',
 '00145ff0jw1dypvr870e2j.jpg',
 '005A1CCejw1es6gd1gzmbj30m80go75c.jpg',
 '005A2Qxrjw1ev1yzustegj30ee0kqjt5.jpg',
 '005A2Qxrjw1ev1yzvrxe9j30fe0qoacl.jpg',
 '005A2Qxrjw1ev1yzwxsw9j30dw0qo40r.jpg',
 '005A2Qxrjw1ev1yzy0qddj30ja0hsgom.jpg',
 '005A3cLbjw1ex2t9erm62j30jq0tmq5q.jpg',
 '005A6GZrjw1et26r0q1dhj30hs0hsabk.jpg',
 '005A7a5Xjw1eqvbzd9ou6j30f00qowfm.jpg',
 '005A7ySTjw1ev2777v12pj30hs0b5gm6.jpg',
 '005AChV9jw1etkv919gp6j30go0m80u4.jpg',
 '005Aejf6jw1ewzuh0kwkij30hv0hsdgc.jpg',
 '005AfNs2jw1ev1ckpwghhj30ku112hdt.jpg',
 '005AfNs2jw1ev1cksesw8j30ku1127wh.jpg',
 '005AfNs2jw1ev1ckur55mj30ku1124qp.jpg',
 '005AfNs2jw1ev1ckxd19dj30ku112b29.jpg',
 '005AG1yAjw1ev2j41jvtlj30ch0ix0ty.jpg',
 '005AG1yAjw1ev2j41wf5lj30k00zlmz7.jpg',
 '005AG1yAjw1ev2j4284z0j30f00qodin.jpg',
 '005AG1yAjw1ev2j42sq3gj30k00vvgny.jpg',
 '005AG8yNjw1ep3h9wub9kj30be0f53z8.jpg',
 '005

In [7]:
# #改为三通道的.jpg图片，只运行一次，不要重复运行
# for file in tqdm(img_list):
#     name = file.split('.')[0]
#     if file.split('.')[-1] != 'jpg':
#         print(file)
#     file = os.path.join(img_path, file)
#     output_path = os.path.join(output_dir, name + ".jpg")
#     # 使用ffmpeg命令行工具将文件转换为三通道的JPG格式
#     subprocess.run(["ffmpeg", "-i", file, "-pix_fmt", "rgb24", output_path])
#     # os.remove(file)
#     # subprocess.run(["ffmpeg", "-y", "-i", file, "-pix_fmt", "rgb24", output_path])

In [8]:
with open('../weibo/tweets/train_nonrumor.txt', 'r', encoding='utf-8') as file:
    train_nonrumor = file.read()
    print(train_nonrumor[:1000], end='')  # 打印文件的前1000个字符

with open('../weibo/tweets/train_rumor.txt', 'r', encoding='utf-8') as file:
    train_rumor = file.read()

with open('../weibo/tweets/test_nonrumor.txt', 'r', encoding='utf-8') as file:
    test_nonrumor = file.read()

with open('../weibo/tweets/test_rumor.txt', 'r', encoding='utf-8') as file:
    test_rumor = file.read()

3929882201725464|null|null|null|1452440393000|true|122|15|56|null|0|null|null|null|微博 weibo.com
http://ww3.sinaimg.cn/large/62b31d36gw1ezuttozq82j20dw0kutax.jpg|http://ww1.sinaimg.cn/large/62b31d36gw1ezuttsqit7j20cn0iyq4k.jpg|http://ww3.sinaimg.cn/large/62b31d36gw1ezuttxjhikj20iw0sc40l.jpg|http://ww2.sinaimg.cn/large/62b31d36gw1ezuttjjyooj20bv0hsdgi.jpg|http://ww2.sinaimg.cn/large/62b31d36gw1ezutu2f15yj20kg0umjtz.jpg|http://ww4.sinaimg.cn/large/62b31d36gw1ezutu4h9tmj20dw0kldhv.jpg|http://ww4.sinaimg.cn/large/62b31d36gw1ezutu99oevj20bu0hsgn0.jpg|http://ww1.sinaimg.cn/large/62b31d36gw1ezutudfrf4j20kg0uo76w.jpg|http://ww3.sinaimg.cn/large/62b31d36gw1ezutuhghhnj20d30kujt5.jpg|null
【每日美图】山居
3902500501958163|null|null|null|1445912087000|true|438|261|687|null|0|null|null|null|iPhone 6
http://ww2.sinaimg.cn/large/a9570da6jw1exfh02r0d5j20p014n79n.jpg|http://ww1.sinaimg.cn/large/a9570da6jw1exfh0421v5j20p015an4r.jpg|http://ww4.sinaimg.cn/large/a9570da6jw1exfh04wxswj20p015479h.jpg|http://ww2.sinai

In [9]:
train_nonrumor_list = train_nonrumor.splitlines()
train_rumor_list = train_rumor.splitlines()

test_nonrumor_list = test_nonrumor.splitlines()
test_rumor_list = test_rumor.splitlines()

# 打印出每个列表的长度除以3的结果，可以看到是3的整数倍
print(len(train_nonrumor_list)/3, len(train_rumor_list)/3, len(test_nonrumor_list)/3, len(test_rumor_list)/3)

3783.0 3748.0 996.0 1000.0


In [10]:
def group_weibo_entries(weibo_list):
    grouped_entries = []
    for i in range(0, len(weibo_list)//3):
        grouped_entries.append(weibo_list[i*3 : (i+1)*3])

    weibo_list = []
    for group in grouped_entries:
        # s = '\n'.join([group[0], group[2]])
        s = group[2]
        weibo_list.append([s, group[1]])

    group_img_list = []
    for group in weibo_list:
        img_list = [ img for img in group[-1].split('|') if img != 'null' ]
        img_list = [ name.split('/')[-1] for name in img_list ]
        group_img_list.append(img_list)

    weibo_list = list(zip([i[0] for i in weibo_list], group_img_list))

    return weibo_list

In [11]:
train_nonrumor_list = group_weibo_entries(train_nonrumor_list)
train_rumor_list = group_weibo_entries(train_rumor_list)
test_nonrumor_list = group_weibo_entries(test_nonrumor_list)
test_rumor_list = group_weibo_entries(test_rumor_list)

for group in test_nonrumor_list[2:5]:
    print(len(group))
    for i in group:
        print(i)
    print()

2
#紫禁城的初雪#意料之外的密匝纷扬。今日入宫，可择一隅，静听碎玉……大家一起#随手拍初雪#吧~
['62aad664jw1exr574z61xj20u01hcanf.jpg', '62aad664jw1exr575cusqj20u01hc7fr.jpg', '62aad664jw1exr575teg9j20u01hck34.jpg', '62aad664jw1exr576bt47j20u01hck36.jpg', '62aad664jw1exr576oe1oj20u01hc4g0.jpg', '62aad664jw1exr5778b8bj20u01hcqek.jpg', '62aad664jw1exr577l36rj20u01hcwyh.jpg', '62aad664jw1exr5781cq7j20u01hcnad.jpg', '62aad664jw1exr578m0tlj20u01hc7fi.jpg']

2
【移民：分配的村子太烂，我们走了】媒体报道，部分被分配到东德小村Sumte的移民，因为失望而决定离开，有移民告诉记者“他们之前告诉我们会去法兰克福，结果却把我们送到了这个荒无人烟的地方”他们觉得太无聊。这个东德小村庄原有居民102人，第一批移民周一到达，750位移民让当地人口剧增了7倍多
['62b31d36gw1expshp70k2j20hm0btmyk.jpg', '62b31d36gw1expshtpnlrj20hm0br76y.jpg', '62b31d36gw1expshufgp2j20hm0c1dh8.jpg', '62b31d36gw1expshwcfsjj20hm0bq40l.jpg', '62b31d36gw1expsi2gfrdj20hm0loq8o.jpg', '62b31d36gw1expsi67migj20hm0od76f.jpg', '62b31d36gw1expsi95pbyj20hm0bijtg.jpg', '62b31d36gw1expsido73zj20hm0ayq4r.jpg', '62b31d36gw1expsiherz4j20hm0azwfj.jpg']

2
【#台媒称贵阳是广西省会#惹怒网友】 网友@虎爷John 爆料称：“近日，@经济学人集团 台湾版报道出现一个低级错误，误将贵阳市称为“广西省省会”，他称《

In [12]:
images_path = r'..\weibo\images'
imgs = os.listdir(images_path)

In [13]:
# 定义函数get_img，用于提取微博帖子的文本和图片数据
def get_img(weibo_list, label):
    arr = []  # 创建一个空列表用于存储结果
    # 遍历weibo_list中的每个元素
    for i in range(len(weibo_list)):
        # 创建一个字典a，包含文本、空图片列表和标签
        a = {'text': weibo_list[i][0], 'img': [], 'label': label}
        # 遍历当前微博帖子的图片名称列表
        for image_name in weibo_list[i][1]:
            # 如果图片格式为gif，则将其转换为jpg
            if image_name.split('.')[-1] == 'gif':
                image_name = image_name.split('.')[0] + ".jpg"
            try:
                # 尝试打开图片文件并读取数据
                with open(os.path.join(images_path, image_name), 'rb') as file:
                    img_data = file.read()
                    # 将图片数据添加到字典a的'img'键对应的列表中
                    a['img'].append(img_data)
            except:
                # 如果打开或读取文件时出错，则忽略该图片
                pass
        # 将字典a添加到结果列表arr中
        arr.append(a)
    # 返回包含所有微博帖子文本和图片数据的列表
    return arr

In [14]:
# 标签0和1分别用于标记谣言和非谣言帖子

# 打开一个文件用于写入，文件名为'train.pkl'
with open('train.pkl', 'wb') as file:
    # 调用get_img函数处理谣言和非谣言的训练集微博帖子
    # 将返回的列表合并，并使用pickle.dump序列化数据，然后写入'train.pkl'文件
    pickle.dump(get_img(train_rumor_list, 0) + get_img(train_nonrumor_list, 1), file)

# 打开一个文件用于写入，文件名为'test.pkl'
with open('test.pkl', 'wb') as file:
    # 调用get_img函数处理非谣言和谣言的测试集微博帖子
    # 将返回的列表合并，并使用pickle.dump序列化数据，然后写入'test.pkl'文件
    pickle.dump(get_img(test_nonrumor_list, 1) + get_img(test_rumor_list, 0), file)

In [15]:
# 打开'test.pkl'文件，用于读取测试数据
with open('test.pkl', 'rb') as file:
    test = pickle.load(file)  # 使用pickle.load函数加载文件内容到变量test

    print(test[-1])  # 打印test列表的最后一个元素

# 使用PIL库打开最后一个条目的第一张图片
image = Image.open(io.BytesIO(test[-1]["img"][0]))
image.show()  # 显示图片

{'text': '【又是南京，又一个彭宇】我正在看：“中国式悲哀，南京法官又要红了！”&nbsp;&nbsp;救死扶伤竟然被判定非法！O网页链接） O中国式悲哀！南京法官又要红了', 'img': [b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00\x01\x00\x01\x00\x00\xff\xfe\x00\x10Lavc60.38.100\x00\xff\xdb\x00C\x00\x08\x16\x16\x1a\x16\x1a\x1e\x1e\x1e\x1e\x1e\x1e#!#%%%####%%%\'\'\'...\'\'\'%%\'\',,..24200.044777BB??MMO^^r\xff\xc4\x00\xc1\x00\x01\x00\x01\x05\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x06\x07\x04\x01\x03\x02\x08\x01\x01\x01\x00\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x04\x03\x02\x05\x06\x07\x10\x00\x02\x01\x02\x02\x04\x04\x10\x0b\x06\x05\x04\x02\x03\x00\x03\x00\x01\x02\x03\x11\x12\x04\x13!\x0514AqQ\xd1\x82RS\xa2a\x91\xd2"\xb1\xa1\x81\xb3\x152\x14\x06r\xc13\xb2\x16B\x92T\x93\xa3#\xe2\xf0db\xe3Cc\xe1s\xd3$\xc25\x83\xf1D%\xc3UtE\x84\x11\x01\x00\x02\x02\x01\x03\x03\x02\x03\x06\x05\x04\x02\x02\x02\x03\x00\x01\x02\x03\x11\x12\x041!QA\x13a2"q\x91R\x81B\x14\x05\x1543b\xa1#\xb1\xc1CS\x82\xd1$r\xe1\xf0DE\xf1\xff\xc0\x00\x11\x08\x08\xc9\x04t\x

In [16]:
# 打开'train.pkl'文件，用于读取训练数据
with open('train.pkl', 'rb') as f:
    data = pickle.load(f)  # 使用pickle.load函数加载文件内容到变量data

print(len(data))  # 打印data列表的长度，即数据条目的总数

a, b = 0, 0  # 初始化计数器a和b，分别用于统计无图和一张图的数据条目数量
# 遍历data列表中的每个元素
for i in data:
    if len(i["img"]) == 0:  # 如果当前条目的图片列表为空
        a += 1  # 无图的计数器加1
    elif len(i["img"]) == 1:  # 如果当前条目的图片列表只有一张图片
        b += 1  # 一张图的计数器加1
        
print(f"无图{a} 一张图{b} 多张图{len(data)-a-b}")

7531
无图1329 一张图4321 多张图1881


In [17]:
with open('test.pkl', 'rb') as f:
    data = pickle.load(f)  # 使用pickle.load函数加载文件内容到变量data

print(len(data))  # 打印data列表的长度，即数据条目的总数

a, b = 0, 0  # 初始化计数器a和b，分别用于统计无图和一张图的数据条目数量
# 遍历data列表中的每个元素
for i in data:
    if len(i["img"]) == 0:  # 如果当前条目的图片列表为空
        a += 1  # 无图的计数器加1
    elif len(i["img"]) == 1:  # 如果当前条目的图片列表只有一张图片
        b += 1  # 一张图的计数器加1
        
print(f"无图{a} 一张图{b} 多张图{len(data)-a-b}")

1996
无图237 一张图1380 多张图379
